# Document Summarisation

## Import Libraries

In [1]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import pandas as pd

## Read File

In [2]:
# Read data
df=pd.read_csv('cleaned_hotelreviews.csv')
df=df.dropna()
df['reviews'] = df['reviews'].str.split(" ")

# testing only first 10 reviews
sentences = df['reviews'].head(10).to_numpy()
sentences = list(sentences)

In [3]:
flat_list = [item for sublist in sentences for item in sublist]

print("Original text:")
for word in flat_list:
    print(word, end=" ")

Original text:
i am so angry that i made this post available via all possible sites i use when planing my trips so no one will make the mistake of booking this place i made my booking via booking com we stayed for nights in this hotel from to july upon arrival we were placed in a small room on the floor of the hotel it turned out that this was not the room we booked i had specially reserved the level duplex room so that we would have a big windows and high ceilings the room itself was ok if you don t mind the broken window that can not be closed hello rain and a mini fridge that contained some sort of a bio weapon at least i guessed so by the smell of it i intimately asked to change the room and after explaining times that i booked a duplex btw it costs the same as a simple double but got way more volume due to the high ceiling was offered a room but only the next day so i had to check out the next day before o clock in order to get the room i waned to not the best way to begin your ho

## Summarisation Algorithm

- Cosine Similarity

In [4]:
# run summarisation

# def read_article(file_name):
#     file = open(file_name, "r")
#     filedata = file.readlines()
#     article = filedata[0].split(". ")
#     sentences = []

#     for sentence in article:
#         print(sentence)
#         sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
#     sentences.pop() 
    
#     return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

def generate_summary(sentences, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []
    
#     # Step 1 - Read text anc split it
#     sentences =  read_article(file_name)
    
    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize texr
    print("\nSummarize Text: \n", ". ".join(summarize_text))

In [5]:
# Execute code
generate_summary(sentences, 2)

Indexes of top ranked_sentence order are  [(0.20676970637011358, ['i', 'am', 'so', 'angry', 'that', 'i', 'made', 'this', 'post', 'available', 'via', 'all', 'possible', 'sites', 'i', 'use', 'when', 'planing', 'my', 'trips', 'so', 'no', 'one', 'will', 'make', 'the', 'mistake', 'of', 'booking', 'this', 'place', 'i', 'made', 'my', 'booking', 'via', 'booking', 'com', 'we', 'stayed', 'for', 'nights', 'in', 'this', 'hotel', 'from', 'to', 'july', 'upon', 'arrival', 'we', 'were', 'placed', 'in', 'a', 'small', 'room', 'on', 'the', 'floor', 'of', 'the', 'hotel', 'it', 'turned', 'out', 'that', 'this', 'was', 'not', 'the', 'room', 'we', 'booked', 'i', 'had', 'specially', 'reserved', 'the', 'level', 'duplex', 'room', 'so', 'that', 'we', 'would', 'have', 'a', 'big', 'windows', 'and', 'high', 'ceilings', 'the', 'room', 'itself', 'was', 'ok', 'if', 'you', 'don', 't', 'mind', 'the', 'broken', 'window', 'that', 'can', 'not', 'be', 'closed', 'hello', 'rain', 'and', 'a', 'mini', 'fridge', 'that', 'containe